1. **Importar pandas y cargar datos**
   ```python
   import pandas as pd
   df_partidos = pd.read_csv("partidos_cebollitas.csv")
   df_partidos.head()
   ```
   - Se importa la librería pandas.
   - Se lee el archivo CSV con datos de partidos y se muestra el inicio del DataFrame.

2. **Crear una nueva columna: diferencia de goles**
   ```python
   df_partidos['diferencia_goles'] = df_partidos['goles_local'] - df_partidos['goles_visitante']
   ```
   - Se calcula la diferencia de goles entre el equipo local y visitante y se agrega como nueva columna.

3. **Mostrar las columnas relevantes**
   ```python
   df_partidos[['goles_local', 'goles_visitante', 'diferencia_goles']].head()
   ```
   - Se muestran las columnas de goles y la diferencia para las primeras filas.

4. **Crear una nueva columna: ratio tiros al arco por posesión local**
   ```python
   df_partidos['ratio_tiros_posesion_local'] = df_partidos['tiros_arco_local'] / df_partidos['posesion_local (%)']
   ```
   - Se calcula el ratio entre tiros al arco y porcentaje de posesión del equipo local.

5. **Mostrar el ratio calculado**
   ```python
   df_partidos[['tiros_arco_local', 'posesion_local (%)', 'ratio_tiros_posesion_local']].head()
   ```
   - Se muestran tiros al arco, posesión y el ratio para las primeras filas.

6. **Visualización: histograma de diferencia de goles**
   ```python
   import matplotlib.pyplot as plt
   import seaborn as sns
   sns.histplot(df_partidos['diferencia_goles'], bins=15, kde=True)
   plt.title('Distribución Diferencia de Goles')
   plt.xlabel('Diferencia de goles (local - visitante)')
   plt.ylabel('Frecuencia')
   plt.show()
   ```
   - Se importa matplotlib y seaborn.
   - Se grafica un histograma de la diferencia de goles para ver su distribución.

7. **Análisis de correlaciones**
   ```python
   corr_vars = ['goles_local', 'goles_visitante', 'diferencia_goles', 'ratio_tiros_posesion_local']
   corr_matrix = df_partidos[corr_vars].corr()
   sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
   plt.title('Correlación entre variables originales y nuevas')
   plt.show()
   ```
   - Se seleccionan variables para analizar correlaciones.
   - Se calcula la matriz de correlación y se muestra como un mapa de calor.

**Resumen:**  
El código carga datos de partidos, crea nuevas características (diferencia de goles y ratio tiros/posesión), visualiza la distribución de la diferencia de goles y analiza correlaciones entre variables originales y nuevas.



GitHub Copilot: Te explico paso a paso el código:

---

### 1. **Carga de datos y creación de variables**

```python
import pandas as pd
df_partidos = pd.read_csv("partidos_cebollitas.csv")
df_partidos['diferencia_goles'] = df_partidos['goles_local'] - df_partidos['goles_visitante']
df_partidos['ratio_tiros_posesion_local'] = df_partidos['tiros_arco_local'] / df_partidos['posesion_local (%)']
df_partidos.head()
```
- Se importa pandas y se carga el archivo CSV con los partidos.
- Se crean dos nuevas columnas:
  - `diferencia_goles`: resta los goles del local y visitante.
  - `ratio_tiros_posesion_local`: tiros al arco local dividido por la posesión local.

---

### 2. **Selección de variables predictoras y objetivo**

```python
from sklearn.feature_selection import SelectKBest, f_regression

X = df_partidos[['posesion_local (%)', 'tiros_arco_local', 'ratio_tiros_posesion_local']]
y = df_partidos['goles_local']
```
- Se seleccionan tres variables para predecir los goles del local (`y`).

---

### 3. **Selección univariada de características**

```python
selector = SelectKBest(score_func=f_regression, k=2)
selector.fit(X, y)
scores = selector.scores_
variables = X.columns
resultados = pd.DataFrame({'Variable': variables, 'Score': scores}).sort_values(by='Score', ascending=False)
resultados
```
- Se usa `SelectKBest` para elegir las 2 variables más relevantes según su relación individual con el objetivo.
- Se muestran los scores de relevancia para cada variable.

---

### 4. **Importancia de variables con árbol de decisión**

```python
from sklearn.tree import DecisionTreeRegressor

modelo_arbol = DecisionTreeRegressor(random_state=42)
modelo_arbol.fit(X, y)
importancia = modelo_arbol.feature_importances_
resultado_importancia = pd.DataFrame({
    'Variable': variables, 
    'Importancia': importancia
}).sort_values(by='Importancia', ascending=False)
resultado_importancia
```
- Se entrena un árbol de decisión para predecir los goles del local.
- Se obtiene la importancia de cada variable según el modelo.

---

### 5. **Visualización de resultados**

```python
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(1, 2, figsize=(12,4))

# Gráfica SelectKBest (análisis univariado)
sns.barplot(x='Score', y='Variable', data=resultados, ax=ax[0], palette='Blues_r')
ax[0].set_title('Selección Univariada (SelectKBest)')

# Gráfica Importancia (árbol decisión)
sns.barplot(x='Importancia', y='Variable', data=resultado_importancia, ax=ax[1], palette='Greens_r')
ax[1].set_title('Importancia Árbol de Decisión')

plt.tight_layout()
plt.show()
```
- Se grafican los resultados de ambos métodos:
  - A la izquierda, la relevancia univariada de cada variable.
  - A la derecha, la importancia según el árbol de decisión.

---

**En resumen:**  
El código carga datos, crea nuevas variables, analiza cuáles son más relevantes para predecir los goles del local usando dos métodos (SelectKBest y árbol de decisión), y visualiza los resultados para comparar qué variables son más importantes según cada técnica.

## Índice

1. ¿Qué es la selección de características?
2. Eliminación de características con baja varianza
3. Selección univariada de características
4. Eliminación recursiva de características (RFE)
5. Selección de características mediante modelos (SelectFromModel)
   - Basada en L1 (Lasso, SVM)
   - Basada en árboles
6. Selección secuencial de características (SFS)
7. Selección de características en un Pipeline

---

## 1. ¿Qué es la selección de características?

**Concepto:**  
Imagina que tienes una canasta llena de manzanas, peras, plátanos y naranjas, pero solo quieres elegir las frutas más dulces para hacer un jugo.  
En machine learning, la selección de características es como elegir solo las frutas (variables) que realmente ayudan a hacer el mejor jugo (modelo).

**Ejemplo técnico:**  
En un dataset con muchas columnas, seleccionamos solo las más útiles para predecir el resultado.

---



## 2. Eliminación de características con baja varianza

**Concepto:**  
Si tienes una caja de crayones y todos son del mismo color, no te ayudan a dibujar cosas diferentes.  
Las características con poca variación no aportan información útil.



**Ejemplo técnico:**  

In [1]:


from sklearn.feature_selection import VarianceThreshold
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)
# Elimina columnas que casi siempre tienen el mismo valor

array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])


## 3. Selección univariada de características

**Concepto:**  
Es como probar cada fruta por separado para ver cuál es la más dulce.  
Seleccionamos las mejores características usando pruebas estadísticas, una por una.

**Ejemplo técnico:**  






In [2]:

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, f_classif
X, y = load_iris(return_X_y=True)
X_new = SelectKBest(f_classif, k=2).fit_transform(X, y)

# Selecciona las 2 mejores columnas
## 4. Eliminación recursiva de características (RFE)

**Concepto:**  
Imagina que tienes muchas piezas de LEGO y vas quitando las menos importantes hasta que tu castillo sigue siendo fuerte pero con menos piezas.  
RFE elimina las características menos útiles una por una.

**Ejemplo técnico:**  





In [3]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(X, y)
# Elimina características menos importantes recursivamente

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RFE(estimator=LogisticRegression(), n_features_to_select=2)

## 5. Selección de características mediante modelos (SelectFromModel)

### Basada en L1 (Lasso, SVM)

**Concepto:**  
Algunos modelos pueden decirte qué piezas son las más importantes para construir tu castillo.  
Si una pieza no ayuda, el modelo la pone en cero y la puedes quitar.










In [4]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
# Selecciona solo las características con coeficiente distinto de cero

### Basada en árboles

**Concepto:**  
Un árbol puede decirte qué ramas son las más fuertes.  
Los modelos de árboles calculan la importancia de cada característica.

**Ejemplo técnico:**  

In [5]:

from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=50).fit(X, y)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
# Selecciona las características más importantes según el árbol



## 6. Selección secuencial de características (SFS)

**Concepto:**  
Es como armar tu equipo de fútbol eligiendo primero al mejor jugador, luego al segundo mejor, y así sucesivamente.  
SFS agrega o elimina características una por una, buscando el mejor equipo.

**Ejemplo técnico:**  




In [6]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
sfs = SequentialFeatureSelector(LinearRegression(), n_features_to_select=2, direction='forward')
sfs.fit(X, y)
# Selecciona características de forma secuencial

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=2)


## 7. Selección de características en un Pipeline

**Concepto:**  
Es como tener una receta donde primero eliges las mejores frutas y luego haces el jugo.  
En machine learning, puedes encadenar la selección de características y el modelo en un solo paso.


In [7]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
  ('classification', RandomForestClassifier())
])
clf.fit(X, y)
# Selecciona características y entrena el modelo en un solo flujo



c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(penalty='l1'))),
                ('classification', RandomForestClassifier())])


## Resumen para niños

- **Seleccionar características** es como elegir solo las mejores frutas para tu jugo.
- Hay muchas formas de elegir: quitar las que no cambian, probar una por una, usar modelos que te dicen cuáles son importantes, o ir agregando las mejores poco a poco.
- Así, tu jugo (modelo) será más rico y fácil de preparar.

¿Te gustaría ejemplos con tus propios datos o alguna técnica en particular?

Similar code found with 1 license type